In [1]:
%reset -f
import sys, os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from wavhandler import *
from utils import *
from keras.layers import Input, Dense
from keras.models import Model
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import logging
logger = logging.getLogger()
logger.propagate = False
logger.setLevel(logging.ERROR)
np.random.seed(0)
from keras.layers import Input, Dense, Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape
from keras.models import Model
from keras import backend as K

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.
Using TensorFlow backend.


In [2]:
dataset = Dataset('LG')
dataset.load(only_names=False)
target_names = dataset.target_classes
X_names = dataset.filenames
y = dataset.y
X = dataset.X
print("X_shape:", X.shape)
print("y_shape:", y.shape)

100%|██████████| 2/2 [00:01<00:00,  1.30it/s]


X_shape: (12746, 5000)
y_shape: (12746,)


In [139]:
%%time
# X = make_df_parallel(names=X_names, setting='psd_dB')
# X = transform_data(dataset.X)

CPU times: user 8 µs, sys: 2 µs, total: 10 µs
Wall time: 18.6 µs


In [140]:
from sklearn.preprocessing import MinMaxScaler

# X_std = StandardScaler(with_std=True).fit_transform(X);  # standardization of data
X_std = MinMaxScaler(feature_range=(0,1)).fit_transform(X)

X, y = shuffle(X_std, y, random_state=3)
#del df

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2018)

In [153]:
input_dim = x_train.shape[1]
encoding_dim = 2
input_img = Input(batch_shape=(None,input_dim,))
encoded = Dense(1024, activation='relu')(input_img)
encoded = Dense(512, activation='relu')(encoded)
# encoded = Dense(16, activation='relu')(encoded)
# encoded = Dense(8, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)
# decoded = Dense(8, activation='relu')(encoded)
# decoded = Dense(16, activation='relu')(decoded)
decoded = Dense(512, activation='relu')(encoded)
decoded = Dense(1024, activation='relu')(encoded)
decoded = Dense(input_dim, activation='softmax')(decoded)
autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adagrad', loss='binary_crossentropy')
print(autoencoder.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        (None, 5000)              0         
_________________________________________________________________
dense_105 (Dense)            (None, 1024)              5121024   
_________________________________________________________________
dense_106 (Dense)            (None, 512)               524800    
_________________________________________________________________
dense_107 (Dense)            (None, 2)                 1026      
_________________________________________________________________
dense_109 (Dense)            (None, 1024)              3072      
_________________________________________________________________
dense_110 (Dense)            (None, 5000)              5125000   
Total params: 10,774,922
Trainable params: 10,774,922
Non-trainable params: 0
________________________________________________________________

In [154]:
# x_train = np.expand_dims(x_train, axis=-1)
# y_train = np.expand_dims(y_train, axis=-1)
# # print(x_train.shape, y_train.shape)

In [155]:
# # # ENCODER

# input_sig = Input(batch_shape=(None,129,1))
# x = Conv1D(64,3, activation='relu', padding='valid')(input_sig)
# x1 = MaxPooling1D(2)(x)
# x2 = Conv1D(32,3, activation='relu', padding='valid')(x1)
# x3 = MaxPooling1D(2)(x2)
# flat = Flatten()(x3)
# encoded = Dense(32,activation = 'relu')(flat)
 
# print("shape of encoded {}".format(K.int_shape(encoded)))
 
# # DECODER 
# x2_ = Conv1D(32, 3, activation='relu', padding='valid')(x3)
# x1_ = UpSampling1D(2)(x2_)
# x_ = Conv1D(64, 3, activation='relu', padding='valid')(x1_)
# upsamp = UpSampling1D(2)(x_)
# flat = Flatten()(upsamp)
# decoded = Dense(129,activation = 'relu')(flat)
# decoded = Reshape((129,1))(decoded)
 
# print("shape of decoded {}".format(K.int_shape(decoded)))
 
# autoencoder = Model(input_sig, decoded)
# autoencoder.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [156]:
history = autoencoder.fit(x_train, x_train,
                epochs=100,
                batch_size=32,
                shuffle=True,
                validation_split=0.2,
                verbose = 1)

Train on 8156 samples, validate on 2040 samples
Epoch 1/100
4224/8156 [==============>...............] - ETA: 6s - loss: 3.9023

KeyboardInterrupt: 

In [ ]:
# use our encoded layer to encode the training input
encoder = Model(input_img, encoded)
encoded_input = Input(shape=(encoding_dim,))
decoder_layer = autoencoder.layers[5]
decoder = Model(encoded_input, decoder_layer(encoded_input))
encoded_data = encoder.predict(x_test)
print(encoded_data.shape)

In [96]:
import matplotlib.pyplot as plt
sns.set()
plt.figure(figsize=(18,14))
sns.scatterplot(encoded_data[:,0], encoded_data[:,1], hue=y_test, style=y_test, alpha=0.8)

NameError: name 'encoded_data' is not defined

<Figure size 1296x1008 with 0 Axes>

In [ ]:
# use Matplotlib (don't ask)
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.plot(x_test[i])#.reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.plot(decoded_imgs[i])#.reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
# 0.0469 loss with bs=128 and encoding_dim=15 